# Imports

In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import Module
from torch import Tensor

import dgl
import dgl.nn.pytorch.conv as dglnn

from torch.utils.data import Dataset, DataLoader, random_split, Subset

import models
import importlib
importlib.reload(models)
from models import FCN1, FCN2, GCN


# Creation of the CustomDataset

In [102]:


class CustomDataset(Dataset):
    def __init__(self, generator_p_set_file, p_load_file, q_load_file, loadings_file):
        self.generator_p_set = pd.read_csv(generator_p_set_file, header=None)
        self.p_load = pd.read_csv(p_load_file, header=None)
        self.q_load = pd.read_csv(q_load_file, header=None)
        self.loadings = pd.read_csv(loadings_file, header=None)

    def __len__(self):
        return self.generator_p_set.shape[1]

    def __getitem__(self, idx):
        generator_p_set_row = self.generator_p_set.iloc[:,idx]
        #print(generator_p_set_row)
        p_load_row = self.p_load.iloc[:,idx]
        q_load_row = self.q_load.iloc[:,idx]
        loadings_row = self.loadings.iloc[:,idx]
        combined_values = torch.tensor(
            list(generator_p_set_row) + list(p_load_row) + list(q_load_row),dtype=torch.float32
        )
        labels = torch.tensor(list(loadings_row),dtype=torch.float32)
        return combined_values, labels

# Define your file paths
generator_p_set_file = "datasets/39BusSystem_full/valid_generators_p.csv"
p_load_file = "datasets/39BusSystem_full/valid_loads_p.csv"
q_load_file = "datasets/39BusSystem_full/valid_loads_q.csv"
loadings_file = "datasets/39BusSystem_full/valid_loadings.csv"

dataset = CustomDataset(generator_p_set_file, p_load_file, q_load_file, loadings_file)


# Creation of the dataloaders with Random Split

In [ ]:
# Create a data loader to iterate over your dataset
batch_size = 100  # Set the batch size according to paper (100)

# Define the split ratios according to paper
train_ratio = 0.84
val_ratio = 0.08
test_ratio = 0.08
torch.random.manual_seed(42)

# Calculate the number of samples for each split
train_size = int(train_ratio * len(dataset))
val_size = int(val_ratio * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset into training, validation, and test sets
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

# Create data loaders for each set
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)


#Create data loaders for GCN neighborhood
dataset_neig = DataLoader(dataset, batch_size=350000, shuffle=False)

len(dataset_neig)
dataset_neig = next(iter(dataset_neig))
input_neig = np.array(dataset_neig[0]).T
output_neig = np.array(dataset_neig[1]).T


In [ ]:
# Get a batch of data from the train_loader
batch_inputs, batch_labels = next(iter(train_loader))

# Print the dimensions
print("Input dimensions:", batch_inputs.shape[1])
print("Output dimensions:", batch_labels.shape[1])

input_dim = batch_inputs.shape[1]
output_dim = batch_labels.shape[1]

# Training, Validation and Testing routines

In [ ]:
# Defining Training, Validation and Testing routines

def train(model, train_loader, valid_loader, optimizer=None, epochs=5, device='cpu', fraction=1, name_file = None):
    if(name_file is not None):
        column_names = ['Epoch', 'Train Loss', 'Valid Loss']
        df = pd.DataFrame(columns = column_names)

    criterion = nn.MSELoss()
    model.to(device)
    
    if optimizer is None:
        optimizer = model.get_optimizer()
    
    dataset_size = len(train_loader.dataset)
    num_train_samples = int(fraction * dataset_size)
    
    for epoch in range(epochs):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            if i * train_loader.batch_size >= num_train_samples:
                break
            
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            for inputs, targets in valid_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                valid_loss += loss.item()
        
        train_loss /= min(len(train_loader), int(num_train_samples / train_loader.batch_size))
        valid_loss /= len(valid_loader)
        
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Valid Loss: {valid_loss:.4f}")
        if name_file is not None:
            new_row = pd.DataFrame([[epoch+1, train_loss, valid_loss]], columns = column_names)
            df = pd.concat([df, new_row], ignore_index=True)
            df.to_csv(name_file, index=False)

def test(model, test_loader, device='cpu', name_file = None):
    criterion = nn.MSELoss()
    model.to(device)
    model.eval()
    
    test_loss = 0.0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
    
    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}")
    if name_file is not None:
        column_names = ['Test Loss']
        df = pd.DataFrame(columns=column_names)
        new_row = pd.DataFrame([[test_loss]], columns = column_names)
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(name_file, index=False)

# Creation of the folders 


In [ ]:
# Folder creations
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        # Create the folder
        os.makedirs(folder_name)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")



# Calculation FOR GCN

In [ ]:
import math

def init_neigh(inputs, outputs, top_k):
    if not isinstance(inputs, list) or not isinstance(inputs[0], list):
        return None
    if not isinstance(outputs, list) or not isinstance(outputs[0], list):
        return None

    n_inputs = inputs
    n_outputs = outputs
    n_standard_deviation_input = calculate_standard_deviations(inputs)
    n_standard_deviation_output = calculate_standard_deviations(outputs)
    n_covariance = calculate_covariances(n_inputs, n_outputs)
    n_correlation = calculate_correlations(n_covariance, n_standard_deviation_input, n_standard_deviation_output)
    n_neighbor = calculate_neighbors(n_correlation)
    n_neig_mat = generate_neighbor_matrix(n_neighbor, top_k)

    return n_neig_mat

def calculate_standard_deviations(data):
    standard_deviations = []
    for x in data:
        mean = sum(x) / len(x)
        sn = 0
        for y in x:
            sn += (y-mean)**2
            sn = math.sqrt(
                sn/len(x)
            )
        standard_deviations.append(sn)
    return standard_deviations

def calculate_covariances(inputs, outputs):
    covariances = []
    for input_data in inputs:
        input_mean = sum(input_data) / len(input_data)
        temp = []
        for output_data in outputs:
            output_mean = sum(output_data) / len(output_data)
            cov = 0
            for idx in range(0, len(input_data)) :
                cov += (input_data[idx]-input_mean)*(output_data[idx]-output_mean)
            cov = cov/len(input_data)
            temp.append(cov)
        covariances.append(temp)
    return covariances

def calculate_correlations(covariances, inputs, outputs):
    correlations = []
    for i in range(len(covariances)):
        temp = []
        for j in range(len(covariances[i])):
            corr = covariances[i][j] / (inputs[i] * outputs[j])
            temp.append(corr)
        correlations.append(temp)
    return correlations

def calculate_neighbors(correlations):
    neighbors = []
    for row in correlations:
        temp = []
        for corr in row:
            count = 0
            for _corr in row :
                if abs(corr) <= abs(_corr) :
                    count += 1
            temp.append(count)
        neighbors.append(temp)
    return neighbors

def generate_neighbor_matrix(neighbors, top_k):
    neighbor_matrix = []
    for row in neighbors:
        temp = []
        for elem in row:
            temp.append(elem <= top_k)
        neighbor_matrix.append(temp)
    return neighbor_matrix

# neighbooring
top_k = output_dim * 1

neighborhood = np.array(init_neigh(input_neig.tolist(), output_neig.tolist(), int(top_k))).T.tolist()
# print('neighborhood : ',neighborhood)

neighborhood_out = np.array(init_neigh(output_neig.tolist(), output_neig.tolist(), 0.2*output_dim)).T.tolist()
print("IMPORTANT SHAPE : ", len(neighborhood))
print(neighborhood_out)
# end neig


# Training routine

In [ ]:


def get_model_and_optimizer(mdl: str):
    if mdl == 'fcn1':
        model = FCN1(in_size=input_dim, out_size=output_dim)
    elif mdl == 'fcn2':
        model = FCN2(in_size=input_dim, out_size=output_dim)
    elif mdl == 'gcn':
        model = GCN(in_features=input_dim, out_features=output_dim, neighborhood=neighborhood, neighborhood_out=neighborhood_out)

    optimizer = model.get_optimizer()
    return model, optimizer

In [ ]:
# Defining the fraction of the dataset to be used for training, test and validation
ds_fraction = 1.0


# Defining the model to train
md = 'gcn'
model, optimizer = get_model_and_optimizer(md)

# Setup device-agnostic code 
# DGL doesn't work with mps
if(md == 'gcn'):
    if torch.cuda.is_available():
        device = "cuda" # NVIDIA GPU
    elif torch.backends.mps.is_available():
        device = "cpu"
else:
    if torch.cuda.is_available():
        device = "cuda" # NVIDIA GPU
    elif torch.backends.mps.is_available():
        device = "mps" # Apple GPU
    else:
        device = "cpu" # Defaults to CPU if NVIDIA GPU/Apple GPU aren't available

print(f"Using device: {device}")

model = model.to(device)

nameFolder =f"modelsTrained/{md}"+datetime.now().strftime("_%d_%m_%Y_%H_%M_%S")
create_folder(nameFolder)
nameFileTraining = nameFolder + "/trainingLog.csv"
nameFileTest = nameFolder + "/testLog.csv"


train(model,train_loader,val_loader,optimizer,250,device,fraction=ds_fraction, name_file = nameFileTraining)
torch.save(model.state_dict(), nameFolder + "/model.pth")
test(model, test_loader, device, name_file = nameFileTest)


# Plot results

In [ ]:
input_file = nameFileTraining

epochs = []
train_losses = []
valid_losses = []

with open(input_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        epochs.append(int(row['Epoch']))
        train_losses.append(float(row['Train Loss']))
        valid_losses.append(float(row['Valid Loss']))

# Plotting
#plt.plot(epochs, train_losses, label='Train Loss')
plt.plot(epochs, valid_losses, label='Validation Loss')

#plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Set logarithmic scale on y-axis
plt.yscale('log')
plt.show()
with open(nameFileTest, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        test_loss = float(row['Test Loss'])
print(f"Test Loss: {test_loss:.4f}")